In [ ]:
!pip install rpy2==3.5.1

In [2]:
# activate R magic
%load_ext rpy2.ipython
import rpy2

In [ ]:
%%R
library(tidyverse) 

In [ ]:
%%R
install.packages("neuralnet") 

In [ ]:
%%R
library(neuralnet)

Dataset contains 100k dimuon events selected from the Mu dataset from Run2010B. Each line corresponds to an event. The events in this derived dataset were selected because of the presence of precisely two muons with invariant mass between 2-110 GeV, one of which is a high-quality "global" muon. 

**Dataset semantics**:


*   Run : The run number of the event.

*   Event : The event number.

*   E : The total energy of the muon (GeV).

*   px,py,pz : The components of the momemtum of the muon (GeV).

*   pt : The transverse momentum of the muon (GeV).
*   eta : The pseudorapidity of the muon.


*   phi : The phi angle of the muon (rad).


*   Q : The charge of the muon.


*   M : The invariant mass of two muons (GeV).


Source: [opendata.cern](http://opendata.cern.ch/record/700)



# Exploring and preparing the data

In [4]:
%%R
df <- read.csv("/content/drive/MyDrive/Dimuon_DoubleMu.csv", stringsAsFactors=FALSE) 
str(df)

'data.frame':	100000 obs. of  21 variables:
 $ Run  : int  165617 165617 165617 165617 165617 165617 165617 165617 165617 165617 ...
 $ Event: int  74601703 75100943 75587682 75660978 75947690 74570517 74697773 74704205 75167029 75206813 ...
 $ type1: chr  "G" "G" "G" "G" ...
 $ E1   : num  9.7 6.2 19.29 7.04 7.28 ...
 $ px1  : num  -9.51 -4.267 -4.212 -6.327 0.103 ...
 $ py1  : num  0.366 0.457 -0.652 -0.269 -5.533 ...
 $ pz1  : num  1.86 -4.48 18.81 3.08 -4.72 ...
 $ pt1  : num  9.52 4.29 4.26 6.33 5.53 ...
 $ eta1 : num  0.195 -0.912 2.191 0.469 -0.774 ...
 $ phi1 : num  3.1 3.04 -2.99 -3.1 -1.55 ...
 $ Q1   : int  -1 -1 -1 -1 -1 -1 -1 1 -1 -1 ...
 $ type2: chr  "G" "G" "G" "G" ...
 $ E2   : num  9.76 9.67 9.82 5.59 7.32 ...
 $ px2  : num  7.328 7.274 4.344 4.475 -0.399 ...
 $ py2  : num  -1.152 -2.821 -0.473 0.849 6.941 ...
 $ pz2  : num  6.35 -5.71 8.8 -3.23 2.28 ...
 $ pt2  : num  7.42 7.8 4.37 4.55 6.95 ...
 $ eta2 : num  0.776 -0.679 1.45 -0.66 0.323 ...
 $ phi2 : num  -0.156 -

The 21 variables in the data frame correspond to the 20 features and one
outcome we expected, although a problem has become apparent. Neural networks
work best when the input data are scaled to a narrow range around zero, and here
we see values ranging anywhere from zero up to over 75,000,000. Typically, the solution to this problem is to rescale the data with a normalizing or
standardization function. I'd like to get a subset of the original dataset first.

In [6]:
%%R
df_subset <- subset(df, select=c('E1','E2','pt1', 'pt2', 'M'))
str(df_subset)

'data.frame':	100000 obs. of  5 variables:
 $ E1 : num  9.7 6.2 19.29 7.04 7.28 ...
 $ E2 : num  9.76 9.67 9.82 5.59 7.32 ...
 $ pt1: num  9.52 4.29 4.26 6.33 5.53 ...
 $ pt2: num  7.42 7.8 4.37 4.55 6.95 ...
 $ M  : num  17.49 11.55 9.16 12.48 14.32 ...


# Taking a closer look at these features

In [7]:
%%R
summary(df_subset)

       E1                 E2                pt1                pt2          
 Min.   :   2.711   Min.   :   2.634   Min.   :  0.2663   Min.   :  0.6828  
 1st Qu.:   8.777   1st Qu.:   6.669   1st Qu.:  5.0190   1st Qu.:  4.9325  
 Median :  13.017   Median :   9.505   Median :  6.9436   Median :  6.8640  
 Mean   :  20.414   Mean   :  14.782   Mean   : 10.0716   Mean   : 10.0077  
 3rd Qu.:  21.071   3rd Qu.:  15.749   3rd Qu.: 10.4396   3rd Qu.: 10.3748  
 Max.   :8684.880   Max.   :1604.970   Max.   :366.4990   Max.   :528.4340  
       M           
 Min.   :  0.3002  
 1st Qu.:  4.4922  
 Median : 12.4110  
 Mean   : 17.6911  
 3rd Qu.: 19.4227  
 Max.   :299.2020  


# Transformation – normalizing numeric data

To normalize these features, we need to create a normalize() function in R. This
function takes a vector x of numeric values, and for each value in x, subtract the minimum value in x and divide by the range of values in x. Finally, the resulting vector is returned. The code for the function is as follows:

In [8]:
%%R
normalize <- function(x) {
 return((x - min(x)) / (max(x) - min(x)))
 }

We can now apply the normalize() function to the numeric features in our data
frame. Rather than normalizing each of the 5 numeric variables individually, we
will use one of R's functions to automate the process. I can use lapply() to apply normalize() to each feature in the subset. 

In [10]:
%%R
df_subset_norm <- as.data.frame(lapply(df_subset, normalize))

To confirm that the normalization worked, we can see that the minimum and
maximum M are now 0 and 1, respectively:

In [11]:
%%R
summary(df_subset_norm$M)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.01402 0.04052 0.05818 0.06398 1.00000 


# Splitting dataset into train and test


In [12]:
%%R
df_subset_train <- df_subset_norm[1:80000, ]
df_subset_test <- df_subset_norm[80001:100000, ] 

We'll use the training dataset to build the neural network and the testing dataset to evaluate how well the model generalizes to future results. As it is easy to overfit a neural network, this step is very important.

# Training a model on the data
I'll begin by training the simplest multilayer feedforward network with only
a single hidden node:

In [19]:
%%R
df_subset_model <- neuralnet(M ~ E1 + E2 + pt1 + pt2, data = df_subset_train, hidden = 1)

# Evaluating model performance
To estimate our model's performance, we can use the `compute()` function to
generate predictions on the testing dataset:

In [20]:
%%R
model_results <- compute(df_subset_model, df_subset_test[1:4]) 

The `compute()` function returns a list with two components: `neurons`, which  stores the neurons for each layer in the network, and `net.results`, which stores the predicted values. I'll want the latter:

In [21]:
%%R
predicted_M <- model_results$net.result

Because this is a numeric prediction problem rather than a classification problem, I cannot use a confusion matrix to examine model accuracy. Instead, I must measure the correlation between my predicted M and the true value. This provides an insight into the strength of the linear association between the two variables. The `cor()` function is used to obtain a correlation between two
numeric vectors:

In [22]:
%%R 
cor(predicted_M, df_subset_test$M) 

         [,1]
[1,] 0.856717


Correlations close to 1 indicate strong linear relationships between two variables. Therefore, the correlation here of about `0.86` indicates a fairly strong relationship. This implies that my model is doing a fairly good job, even with only a single hidden node.